In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U langchain
!pip install -q einops
!pip install -q datasets
# !pip install sentencepiece #optional, only for some models on old LLama 

In [2]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from datasets import load_dataset

import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

from langchain.prompts import PromptTemplate

from IPython.display import Markdown, display

In [3]:
from datasets import load_dataset,concatenate_datasets, Dataset

mmlu   = load_dataset("kz-transformers/mmlu-translated-kk")["validation"]
const  = load_dataset("kz-transformers/kazakh-constitution-mc")["test"]
dastur = load_dataset("kz-transformers/kazakh-dastur-mc")["test"]
ent    = load_dataset("kz-transformers/kazakh-unified-national-testing-mc")


In [4]:
template_mmlu = """Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.\n\n

Cұрақ: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n

Жауап:"""

prompt_mmlu = PromptTemplate(template=template_mmlu, input_variables=['prompt', 'a', 'b', 'c', 'd'])

template_ent = """Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.\n\n

Cұрақ: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n
E) {e}\n
F) {f}\n
G) {g}\n
H) {h}\n

Жауап:"""

prompt_ent = PromptTemplate(template=template_ent, input_variables=['prompt', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])

In [5]:
def format_text_mmlu(example):
    text = prompt_mmlu.format(prompt=example['Question'],
                         a=example['Option A'],
                         b=example['Option B'],
                         c=example['Option C'],
                         d=example['Option D'])
    return {"text": text}

def format_text_ent(example):
    text = prompt_ent.format(prompt=example['question'],
                         a=example['A'],
                         b=example['B'],
                         c=example['C'],
                         d=example['D'],
                         e=example['E'],
                         f=example['F'],
                         g=example['G'],
                         h=example['H'])
    return {"text": text}

In [6]:
mmlu = mmlu.map(format_text_mmlu)
const = const.map(format_text_mmlu)
dastur = dastur.map(format_text_mmlu)
ent = ent.map(format_text_ent)

In [7]:
print(mmlu['text'][0])
print("*"*50)
print(const['text'][0])
print("*"*50)
print(dastur['text'][0])
print("*"*50)
print(ent['biology']['text'][67])

Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.



Cұрақ: 18 арқылы жасалған Z_24 циклдік топшасының реті бар

A) 4

B) 8

C) 12

D) 6


Жауап:
**************************************************
Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.



Cұрақ: Қазақстан Республикасы өзін қандай мемлекет ретінде орнықтырады?

A) Автократиялық

B) Демократиялық, зайырлы, құқықтық және әлеуметтік

C) Тоталитарлық

D) Монархиялық


Жауап:
**************************************************
Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан м

In [8]:
mmlu = mmlu.to_pandas()
const = const.to_pandas()
dastur = dastur.to_pandas()
ent = pd.concat([ent[i].to_pandas() for i in ent])
ent['idx'] = ent.index.astype(str)+"-"+ent['subject']
mmlu['idx'] = mmlu.index.astype(str)+"-mmlu"
const['idx'] = const.index.astype(str)+"-const"
dastur['idx'] = dastur.index.astype(str)+"-dastur"
ent = ent.rename({'correct_answer': 'answer'},axis=1)
mmlu = mmlu.rename({'Correct Answer': 'answer'},axis=1)
const = const.rename({'Correct Answer': 'answer'},axis=1)
dastur = dastur.rename({'Correct Answer': 'answer'},axis=1)
columns = ["answer", "text", "idx"]
mmlu_like = pd.concat([mmlu[columns], const[columns], dastur[columns]])

In [9]:
from datasets import Dataset
ent_ds = Dataset.from_pandas(ent[columns])
mmlu_like_ds = Dataset.from_pandas(mmlu_like)

In [10]:
#model_id = "TilQazyna/llama-kaz-instruct-8B-1"
#model_id = "Qwen/Qwen2.5-7B-Instruct"
#model_id = "google/gemma-2-2b-it"
#model_id = "google/gemma-2-9b-it"
#model_id = "AmanMussa/llama2-kazakh-7b"
#model_id = "LLaMAX/LLaMAX3-8B"
#model_id = "IrbisAI/Irbis-7b-v0.1"# at the end of all experiments
model_id = "AmanMussa/llama2-kazakh-7b-ver2"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True
).to('cuda')

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

In [ ]:
model.eval()

In [ ]:
def get_ans(text, mode="mmlu"):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits[0, -1]

    del inputs
    torch.cuda.empty_cache()

    options_list = [(logits[tokenizer(' A').input_ids[-1]], 'A'),
                    (logits[tokenizer(' B').input_ids[-1]], 'B'),
                    (logits[tokenizer(' C').input_ids[-1]], 'C'),
                    (logits[tokenizer(' D').input_ids[-1]], 'D')]
    if mode == "ent":
        options_list= options_list+[(logits[tokenizer(' E').input_ids[-1]], 'E'),
                                    (logits[tokenizer(' F').input_ids[-1]], 'F'),
                                    (logits[tokenizer(' G').input_ids[-1]], 'G'),
                                    (logits[tokenizer(' H').input_ids[-1]], 'H')]
    options_list = sorted(options_list, reverse=True)
    return options_list[0]


In [ ]:
from tqdm.auto import tqdm
answers_mmlu = []
bar = tqdm(enumerate(mmlu_like_ds), total=len(mmlu_like_ds))
for i, data in bar:
    ans_list = get_ans(data['text'])
    predict = ans_list[1]
    answer = data['answer']
    if predict == answer:
        acc = 1
    else:
        acc = 0
    idx = data['idx']
    answers_mmlu.append({'idx': idx, 'answer': answer, 'predict': predict, 'acc': acc})


In [ ]:
from tqdm.auto import tqdm
answers_ent = []
bar = tqdm(enumerate(ent_ds), total=len(ent_ds))
for i, data in bar:
    ans_list = get_ans(data['text'], mode="ent")
    predict = ans_list[1]
    answer = data['answer']
    if predict == answer:
        acc = 1
    else:
        acc = 0
    idx = data['idx']
    answers_ent.append({'idx': idx, 'answer': answer, 'predict': predict, 'acc': acc})

In [ ]:
answers_mmlu = pd.DataFrame(answers_mmlu)
answers_ent = pd.DataFrame(answers_ent)

In [ ]:
df = pd.concat((answers_mmlu,answers_ent))

In [ ]:
df['dataset'] = df.idx.apply(lambda x: x.replace(x.split('-')[0]+"-",""))
final = pd.merge(df.groupby('dataset').acc.sum(), df.groupby('dataset').acc.count(), on=['dataset'])
final['acc'] = final.acc_x/final.acc_y

In [ ]:
final

In [ ]:
final['acc'].head(10).mean()

In [ ]:
name = "_".join(model_id.split("/"))
final.to_csv(f"final-{name}.csv")
df.to_csv(f"df-{name}.csv")